In [3]:
import pandas as pd

# CSV 파일 로드
p_result = pd.read_csv("/mnt/W-PANCDR/src/logs/full_CV_PANCDR/GDSC_results.csv")
w_result = pd.read_csv("/mnt/W-PANCDR/src/logs/full_CV_WANCDR/GDSC_results.csv")
# 'Best_params' 기준으로 groupby 후, 'Test_AUC' 평균 계산
p_grouped = p_result.groupby('Best_params')['Test_AUC'].mean()
w_grouped = w_result.groupby('Best_params')['Test_AUC'].mean()
# 평균 Test_AUC가 높은 상위 5개의 Best_params 선택
p_top_5 = p_grouped.sort_values(ascending=False).head(5)
w_top_5 = w_grouped.sort_values(ascending=False).head(5)
p_top_5.to_csv("/mnt/W-PANCDR/src/logs/full_CV_PANCDR/top_5_results.csv")
w_top_5.to_csv("/mnt/W-PANCDR/src/logs/full_CV_WANCDR/top_5_results.csv")
# 결과 출력
print(p_top_5)
print(w_top_5)


Best_params
{'nz': 100, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.01, 'batch_size': [128, 14]}    0.804502
{'nz': 256, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.001, 'lam': 0.1, 'batch_size': [128, 14]}      0.802544
{'nz': 128, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.001, 'lam': 1, 'batch_size': [128, 14]}        0.802317
{'nz': 100, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.1, 'batch_size': [256, 28]}     0.802222
{'nz': 128, 'd_dim': 100, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.01, 'batch_size': [128, 14]}    0.801734
Name: Test_AUC, dtype: float64
Best_params
{'nz': 128, 'd_dim': 100, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.0001, 'batch_size': [128, 14]}    0.832946
{'nz': 128, 'd_dim': 128, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.0001, 'batch_size': [128, 14]}    0.831610
{'nz': 100, 'd_dim': 256, 'lr': 0.0001, 'lr_adv': 0.0001, 'lam': 0.0001, 'batch_size': [128, 14]}    0.831136
{'nz': 100, 'd_dim': 256, 'lr': 0.001, 'lr_adv': 0.001, 'lam': 0.0001, 'bat

In [7]:
import pandas as pd

w_result = pd.read_csv('WANCDR_TCGA_100train_results.csv')
w_result.describe()

,Iteration,Accuracy,AUC,F1,Recall,Precision
count,46.000000,46.000000,46.000000,46.000000,46.000000,46.000000
mean,22.500000,0.668854,0.719236,0.670157,0.699690,0.649899
std,13.422618,0.013514,0.014553,0.029423,0.079676,0.026954
min,0.000000,0.621622,0.672079,0.596491,0.507740,0.589421
25%,11.250000,0.661036,0.712377,0.645265,0.639319,0.634299
50%,22.500000,0.671171,0.718004,0.679065,0.713622,0.649841
75%,33.750000,0.675676,0.730569,0.692970,0.760836,0.665141
max,45.000000,0.699700,0.750011,0.726776,0.842105,0.745455


In [9]:
import pandas as pd

w_result = pd.read_csv('PANCDR_TCGA_100train_results.csv')
w_result.describe()

,Iteration,Accuracy,AUC,F1,Recall,Precision
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,4.000000,0.673507,0.715302,0.676420,0.706570,0.652658
std,2.738613,0.017236,0.022929,0.023990,0.063482,0.026914
min,0.000000,0.650150,0.682432,0.626427,0.594427,0.616471
25%,2.000000,0.663664,0.696161,0.662595,0.671827,0.627551
50%,4.000000,0.668168,0.710910,0.685714,0.696594,0.653614
75%,6.000000,0.686186,0.728330,0.691617,0.755418,0.663664
max,8.000000,0.702703,0.749244,0.700535,0.811146,0.703583
